In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

## Libraries

In [1]:
!apt-get install -y rubberband-cli
!pip install pyrubberband librosa soundfile pydub openai-whisper openai moviepy
!pip install TTS

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  rubberband-cli
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 87.5 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 rubberband-cli amd64 2.0.0-2 [87.5 kB]
Fetched 87.5 kB in 0s (255 kB/s)
Selecting previously unselected package rubberband-cli.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../rubberband-cli_2.0.0-2_amd64.deb ...
Unpacking rubberband-cli (2.0.0-2) ...
Setting up rubberband-cli (2.0.0-2) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 13.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━

In [49]:
import os
import subprocess
import re
import time
import torch
from pydub import AudioSegment
import whisper
from openai import OpenAI
import librosa
import pyrubberband
import soundfile as sf
from moviepy.editor import VideoFileClip, AudioFileClip
from TTS.api import TTS
import openai
from moviepy.editor import VideoFileClip, AudioFileClip

In [32]:
# Extracting Audio from Video.

def extract_audio(video_path, audio_path):
    """Extracts audio from a video file using FFmpeg."""
    if os.path.exists(audio_path):
        os.remove(audio_path)
    result = subprocess.run(
        ["ffmpeg", "-y", "-i", video_path, "-q:a", "0", "-map", "a", audio_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        check=True
    )

# Define paths (upload your video to Colab first)
video_path = "/content/self-introduction.mp4"
audio_path = "/content/self-introduction_audio.wav"
extract_audio(video_path, audio_path)
print("Audio extracted successfully!")

Audio extracted successfully!


In [33]:
# Original Audio Duration

original_audio = AudioSegment.from_wav(audio_path)
total_duration_ms = len(original_audio)
print(f"Original audio duration: {total_duration_ms} ms")

Original audio duration: 67570 ms


In [34]:
# Transcribe audio using whisper

model = whisper.load_model("base")
transcript = model.transcribe(
    audio=audio_path,
    language="en",
    word_timestamps=True
)
print(transcript['text'])

  checkpoint = torch.load(fp, map_location=device)



 Hi, my name is Regente Lentino, you can call me John. I'm 24 years old. I have a three-year work experience with the BPO industry doing inbound and outbound calling. First I served as a technical consultant. We're in my multi-tasking skills has been definitely enhanced. Then eventually I switched to customer service. We're in our integrity place a very big role. Definitely sales must always be on point ahead of the target and at the same time making sure that each and every customer will always be satisfied with my assistance. And on top of all these skills I also have a talent with doing graphic designs, creating flyers, business cards and social media advertisements which will really be helpful for clients marketing needs. And I can assure you that I'm not just gonna be another number on your payroll but definitely a good addition to your company. Once again this is Regente Lentino thank you and goodbye.


In [35]:
# Detecting starting silence/pauses in video

def detect_leading_silence(audio_path, silence_threshold=-40, chunk_size=10):
    """Detects the duration of silence at the beginning of an audio file."""
    audio = AudioSegment.from_wav(audio_path)
    leading_silence = 0
    for i in range(0, len(audio), chunk_size):
        chunk = audio[i:i + chunk_size]
        if chunk.dBFS > silence_threshold:
            break
        leading_silence += chunk_size
    return leading_silence / 1000.0

leading_silence_sec = detect_leading_silence(audio_path)
print(f"Leading silence detected: {leading_silence_sec} seconds")

Leading silence detected: 0.85 seconds


In [78]:
# Transcription Into sentences

sentences = []
current_sentence = []
sentence_start = None

for segment in transcript['segments']:
    for word in segment['words']:
        adjusted_start = word['start'] + leading_silence_sec
        adjusted_end = word['end'] + leading_silence_sec
        if not current_sentence:
            sentence_start = adjusted_start
        current_sentence.append(word['word'])
        if word['word'].strip().endswith(('.', '?', '!')):
            sentence_end = adjusted_end
            sentence_text = ' '.join(current_sentence)
            sentences.append({
                'text': sentence_text,
                'start': sentence_start,
                'end': sentence_end
            })
            current_sentence = []
            sentence_start = None
if current_sentence:
    sentence_end = adjusted_end
    sentence_text = ' '.join(current_sentence)
    sentences.append({
        'text': sentence_text,
        'start': sentence_start,
        'end': sentence_end
    })

print("Transcribed sentences with timings:")
for sentence in sentences:
    print(f"{sentence['text']} [{sentence['start']} - {sentence['end']}]")

Transcribed sentences with timings:
 Hi,  my  name  is  Regente  Lentino,  you  can  call  me  John. [0.85 - 5.1899999999999995]
 I'm  24  years  old. [5.989999999999999 - 7.13]
 I  have  a  three -year  work  experience  with  the  BPO  industry  doing  inbound  and  outbound  calling. [7.39 - 13.29]
 First  I  served  as  a  technical  consultant. [14.41 - 17.03]
 We're  in  my  multi -tasking  skills  has  been  definitely  enhanced. [17.57 - 20.770000000000003]
 Then  eventually  I  switched  to  customer  service. [21.43 - 24.110000000000003]
 We're  in  our  integrity  place  a  very  big  role. [24.39 - 27.270000000000003]
 Definitely  sales  must  always  be  on  point  ahead  of  the  target  and  at  the  same  time  making  sure  that  each  and  every  customer  will  always  be  satisfied  with  my  assistance. [28.41 - 38.870000000000005]
 And  on  top  of  all  these  skills  I  also  have  a  talent  with  doing  graphic  designs,  creating  flyers,  business  cards  an

In [79]:
sentences

[{'text': ' Hi,  my  name  is  Regente  Lentino,  you  can  call  me  John.',
  'start': 0.85,
  'end': 5.1899999999999995},
 {'text': " I'm  24  years  old.", 'start': 5.989999999999999, 'end': 7.13},
 {'text': ' I  have  a  three -year  work  experience  with  the  BPO  industry  doing  inbound  and  outbound  calling.',
  'start': 7.39,
  'end': 13.29},
 {'text': ' First  I  served  as  a  technical  consultant.',
  'start': 14.41,
  'end': 17.03},
 {'text': " We're  in  my  multi -tasking  skills  has  been  definitely  enhanced.",
  'start': 17.57,
  'end': 20.770000000000003},
 {'text': ' Then  eventually  I  switched  to  customer  service.',
  'start': 21.43,
  'end': 24.110000000000003},
 {'text': " We're  in  our  integrity  place  a  very  big  role.",
  'start': 24.39,
  'end': 27.270000000000003},
 {'text': ' Definitely  sales  must  always  be  on  point  ahead  of  the  target  and  at  the  same  time  making  sure  that  each  and  every  customer  will  always  be  sa

In [103]:
import re
from openai import OpenAI

openai_api_key = "YOUR_API_KEY"
client = OpenAI(api_key=openai_api_key)

def translate_to_hindi(sentences):
    """Enhanced translation with strict structure constraints."""
    prompt = """Translate these English sentences to Hindi with:
1. EXACT same sentence count
2. Preserve original punctuation
3. Match syllable count where possible
4. Use phonetic spellings for technical terms
5. Maintain original sentence structure

Examples:
"""
    for i, sent in enumerate(sentences):
        prompt += f"{i+1}. {sent['text']}\n"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a professional translator specializing in audiovisual synchronization."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=2000
    )

    translated_text = response.choices[0].message.content.strip()

    # Extract and clean translations from the numbered output
    translations = re.findall(r'^\s*\d+\.\s*(.*?)(?=\n\s*\d+\.|$)', translated_text, re.MULTILINE | re.DOTALL)
    translations = [clean_translation(text) for text in translations]

    if len(translations) != len(sentences):
        raise ValueError(f"Mismatch in translations: expected {len(sentences)}, got {len(translations)}.\nRaw output:\n{translated_text}")

    return translations

def clean_translation(output_text):
    """ Cleans up translation output by removing unwanted spaces or numbers. """
    output_text = re.sub(r'\d+\.', '', output_text)  # Remove numbering if left over
    output_text = re.sub(r'\s+', ' ', output_text).strip()  # Normalize spaces
    return output_text

translated_texts = translate_to_hindi(sentences)

translated_text = " ".join(translated_texts).replace("'", "").strip()
print(translated_text)


नमस्ते, मेरा नाम रेजेंटे लेंटिनो है, आप मुझे जॉन कह सकते हैं। मैं 24 साल का हूँ। मेरे पास BPO इंडस्ट्री में तीन साल का कामयाबी से अनुभव है, इनबाउंड और आउटबाउंड कॉलिंग करते हुए। पहले मैंने एक तकनीकी सलाहकार के रूप में सेवा प्रदान की। हमारे मल्टी-टास्किंग कौशल में निश्चित रूप से सुधार हुआ है। फिर आखिरकार मैंने कस्टमर सर्विस में स्विच किया। हमारे ईमानदारी का बहुत बड़ा योगदान है। निश्चित रूप से बिक्री हमेशा लक्ष्य के आगे होनी चाहिए और उसी समय सुनिश्चित करना चाहिए कि प्रत्येक ग्राहक मेरी सहायता से हमेशा संतुष्ट रहेगा। और इन सभी कौशलों के ऊपर मेरे पास ग्राफिक डिजाइन करने की भी एक प्रतिभा है, फ्लायर्स, बिजनेस कार्ड्स और सोशल मीडिया विज्ञापन बनाने में जो वास्तव में ग्राहकों की विपणन आवश्यकताओं के लिए सहायक होगा। और मैं आपको यह आश्वासन दे सकता हूँ कि मैं आपके पेरोल पर एक और संख्या नहीं होऊंगा, बल्कि निश्चित रूप से आपकी कंपनी में एक अच्छा योगदान होऊंगा। फिर से यह रेजेंटे लेंटिनो है, धन्यवाद और अलविदा।


In [104]:
translated_text = "नमस्ते, मेरा नाम रेजेंटे लेंटिनो है, आप मुझे जॉन कह सकते हैं। मैं 24 साल का हूँ। मेरे पास BPO इंडस्ट्री में तीन साल का कामयाबी से अनुभव है, इनबाउंड और आउटबाउंड कॉलिंग करते हुए। पहले मैंने एक तकनीकी सलाहकार के रूप में सेवा प्रदान की। हमारे मल्टी-टास्किंग कौशल में निश्चित रूप से सुधार हुआ है। फिर आखिरकार मैंने कस्टमर सर्विस में स्विच किया। हमारे ईमानदारी का बहुत बड़ा योगदान है। निश्चित रूप से बिक्री हमेशा लक्ष्य के आगे होनी चाहिए और उसी समय सुनिश्चित करना चाहिए कि प्रत्येक ग्राहक मेरी सहायता से हमेशा संतुष्ट रहेगा। और इन सभी कौशलों के ऊपर मेरे पास ग्राफिक डिजाइन करने की भी एक प्रतिभा है, फ्लायर्स, बिजनेस कार्ड्स और सोशल मीडिया विज्ञापन बनाने में जो वास्तव में ग्राहकों की विपणन आवश्यकताओं के लिए सहायक होगा। और मैं आपको यह आश्वासन दे सकता हूँ कि मैं आपके पेरोल पर एक और संख्या नहीं होऊंगा, बल्कि निश्चित रूप से आपकी कंपनी में एक अच्छा योगदान होऊंगा। फिर से यह रेजेंटे लेंटिनो है, धन्यवाद और अलविदा।"

In [105]:
# Pair hindi sentences with timings

def pair_hindi_sentences_with_timings(translated_text, sentences):
    """Pairs translated Hindi sentences with original timings."""
    hindi_sentences = [s.strip() for s in translated_text.split('।') if s.strip()]
    if len(hindi_sentences) != len(sentences):
        raise ValueError(f"Sentence count mismatch: {len(hindi_sentences)} Hindi vs {len(sentences)} English")
    return [{'text': hindi_sentence + '।', 'start': sentences[i]['start'], 'end': sentences[i]['end']}
            for i, hindi_sentence in enumerate(hindi_sentences)]

hindi_sentence_timings = pair_hindi_sentences_with_timings(translated_text, sentences)
print("Hindi sentences with timings:")
for sentence in hindi_sentence_timings:
    print(f"{sentence['text']} [{sentence['start']} - {sentence['end']}]")

Hindi sentences with timings:
नमस्ते, मेरा नाम रेजेंटे लेंटिनो है, आप मुझे जॉन कह सकते हैं। [0.85 - 5.1899999999999995]
मैं 24 साल का हूँ। [5.989999999999999 - 7.13]
मेरे पास BPO इंडस्ट्री में तीन साल का कामयाबी से अनुभव है, इनबाउंड और आउटबाउंड कॉलिंग करते हुए। [7.39 - 13.29]
पहले मैंने एक तकनीकी सलाहकार के रूप में सेवा प्रदान की। [14.41 - 17.03]
हमारे मल्टी-टास्किंग कौशल में निश्चित रूप से सुधार हुआ है। [17.57 - 20.770000000000003]
फिर आखिरकार मैंने कस्टमर सर्विस में स्विच किया। [21.43 - 24.110000000000003]
हमारे ईमानदारी का बहुत बड़ा योगदान है। [24.39 - 27.270000000000003]
निश्चित रूप से बिक्री हमेशा लक्ष्य के आगे होनी चाहिए और उसी समय सुनिश्चित करना चाहिए कि प्रत्येक ग्राहक मेरी सहायता से हमेशा संतुष्ट रहेगा। [28.41 - 38.870000000000005]
और इन सभी कौशलों के ऊपर मेरे पास ग्राफिक डिजाइन करने की भी एक प्रतिभा है, फ्लायर्स, बिजनेस कार्ड्स और सोशल मीडिया विज्ञापन बनाने में जो वास्तव में ग्राहकों की विपणन आवश्यकताओं के लिए सहायक होगा। [40.13 - 54.050000000000004]
और मैं आपको यह आश्वासन दे

In [106]:
# Step 5: Generate TTS Audio
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

hindi_numbers = {
    "0": "शून्य", "1": "एक", "2": "दो", "3": "तीन", "4": "चार", "5": "पांच", "6": "छह",
    "7": "सात", "8": "आठ", "9": "नौ", "10": "दस", "11": "ग्यारह", "12": "बारह",
    "13": "तेरह", "14": "चौदह", "15": "पंद्रह", "16": "सोलह", "17": "सत्रह",
    "18": "अठारह", "19": "उन्नीस", "20": "बीस", "21": "इक्कीस", "22": "बाईस",
    "23": "तेईस", "24": "चौबीस", "25": "पच्चीस", "26": "छब्बीस", "27": "सत्ताईस",
    "28": "अट्ठाईस", "29": "उनतीस", "30": "तीस", "40": "चालीस", "50": "पचास",
    "60": "साठ", "70": "सत्तर", "80": "अस्सी", "90": "नब्बे", "100": "सौ"
}

def convert_numbers(match):
    """Converts numbers to Hindi words."""
    num = match.group()
    return hindi_numbers.get(num, num)

def preprocess_hindi_text(text):
    """Preprocesses Hindi text by converting numbers and removing periods."""
    text = re.sub(r'\d+', convert_numbers, text)
    text = re.sub(r'\.\s*', ' ', text)
    return text

trash_dir = "/content/trash"
os.makedirs(trash_dir, exist_ok=True)
output_files = []
for i, timing in enumerate(hindi_sentence_timings):
    processed_text = preprocess_hindi_text(timing['text'])
    output_path = os.path.join(trash_dir, f"output_{i}.wav")
    tts.tts_to_file(
        text=processed_text,
        file_path=output_path,
        speaker_wav="/content/input_neeraj.m4a",
        language="hi",
        speed=0.1
    )
    output_files.append(output_path)
print("Generated audio files:", output_files)

  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")



 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


  self.speakers = torch.load(speaker_file_path)

  return torch.load(f, map_location=map_location, **kwargs)



 > Text splitted to sentences.
['नमस्ते, मेरा नाम रेजेंटे लेंटिनो है, आप मुझे जॉन कह सकते हैं।']
 > Processing time: 2.3786957263946533
 > Real-time factor: 0.3962456240707883
 > Text splitted to sentences.
['मैं चौबीस साल का हूँ।']
 > Processing time: 3.41571307182312
 > Real-time factor: 0.40024484117900155
 > Text splitted to sentences.
['मेरे पास BPO इंडस्ट्री में तीन साल का कामयाबी से अनुभव है, इनबाउंड और आउटबाउंड कॉलिंग करते हुए।']
 > Processing time: 3.0552897453308105
 > Real-time factor: 0.39274052609682153
 > Text splitted to sentences.
['पहले मैंने एक तकनीकी सलाहकार के रूप में सेवा प्रदान की।']
 > Processing time: 2.1355342864990234
 > Real-time factor: 0.38476051622192003
 > Text splitted to sentences.
['हमारे मल्टी-टास्किंग कौशल में निश्चित रूप से सुधार हुआ है।']
 > Processing time: 2.2374186515808105
 > Real-time factor: 0.3853821496325215
 > Text splitted to sentences.
['फिर आखिरकार मैंने कस्टमर सर्विस में स्विच किया।']
 > Processing time: 1.9852280616760254
 > Real-time

In [107]:
# Step 6: Adjust Audio Clips for Synchronization
def adjust_audio_clips_with_timings(audio_paths, timings, output_folder):
    """Adjusts each audio clip to match its target duration with speed adjustment and trimming."""
    os.makedirs(output_folder, exist_ok=True)
    adjusted_paths = []
    for i, audio_path in enumerate(audio_paths):
        audio = AudioSegment.from_wav(audio_path)
        current_duration_ms = len(audio)
        target_duration_ms = (timings[i]['end'] - timings[i]['start']) * 1000
        target_duration_sec = target_duration_ms / 1000.0
        speed_factor = current_duration_ms / target_duration_ms
        output_path = os.path.join(output_folder, f"adjusted_clip_{i}.wav")

        atempo_filters = []
        remaining_factor = speed_factor
        while remaining_factor > 2.0:
            atempo_filters.append("atempo=2.0")
            remaining_factor /= 2.0
        while remaining_factor < 0.5:
            atempo_filters.append("atempo=0.5")
            remaining_factor /= 0.5
        if remaining_factor != 1.0:
            atempo_filters.append(f"atempo={remaining_factor:.4f}")

        atrim_filter = f"atrim=duration={target_duration_sec}"
        fade_start = max(0, target_duration_sec - 0.01)
        afade_filter = f"afade=t=out:st={fade_start}:d=0.01"

        filter_list = atempo_filters + [atrim_filter, afade_filter]
        filter_str = ",".join(filter_list)

        audio.export(output_path, format="wav", parameters=["-filter:a", filter_str])
        adjusted_paths.append(output_path)
    return adjusted_paths

output_folder = "/content/adjusted_clips"
adjusted_audio_paths = adjust_audio_clips_with_timings(output_files, hindi_sentence_timings, output_folder)
print("Adjusted audio file paths:", adjusted_audio_paths)

Adjusted audio file paths: ['/content/adjusted_clips/adjusted_clip_0.wav', '/content/adjusted_clips/adjusted_clip_1.wav', '/content/adjusted_clips/adjusted_clip_2.wav', '/content/adjusted_clips/adjusted_clip_3.wav', '/content/adjusted_clips/adjusted_clip_4.wav', '/content/adjusted_clips/adjusted_clip_5.wav', '/content/adjusted_clips/adjusted_clip_6.wav', '/content/adjusted_clips/adjusted_clip_7.wav', '/content/adjusted_clips/adjusted_clip_8.wav', '/content/adjusted_clips/adjusted_clip_9.wav', '/content/adjusted_clips/adjusted_clip_10.wav']


In [108]:
# Step 7: Create Synchronized Audio with Offset
def create_synchronized_audio(adjusted_audio_paths, timings, total_duration_ms, output_path, start_offset_ms=200):
    """
    Creates synchronized audio by overlaying adjusted clips at adjusted start times.
    Applies an offset to all clips except the first one to start earlier.
    """
    synchronized_audio = AudioSegment.silent(duration=total_duration_ms)
    for i, audio_path in enumerate(adjusted_audio_paths):
        clip = AudioSegment.from_wav(audio_path)
        # Apply offset only to clips after the first one (i > 0)
        offset = start_offset_ms if i > 0 else 0
        start_ms = max(0, int(timings[i]['start'] * 1000) - offset)  # Prevent negative start times
        synchronized_audio = synchronized_audio.overlay(clip, position=start_ms)
    synchronized_audio.export(output_path, format="wav")
    return output_path

synchronized_audio_path = "/content/synchronized_output.wav"
final_audio = create_synchronized_audio(adjusted_audio_paths, hindi_sentence_timings, total_duration_ms, synchronized_audio_path, start_offset_ms=400)
print(f"Synchronized audio saved at: {final_audio}")

Synchronized audio saved at: /content/synchronized_output.wav


In [109]:
# Step 8: Replace Audio in Video
video_clip = VideoFileClip(video_path)
new_audio_clip = AudioFileClip(synchronized_audio_path)
final_video = video_clip.set_audio(new_audio_clip)
final_video_output = "/content/self-introduction_final.mp4"
final_video.write_videofile(final_video_output, codec="libx264", audio_codec="aac")
print(f"Final video saved to: {final_video_output}")

100%|██████████| 7.75M/7.75M [2:06:23<00:00, 59.6MiB/s]

Moviepy - Building video /content/self-introduction_final.mp4.
MoviePy - Writing audio in self-introduction_finalTEMP_MPY_wvf_snd.mp4



chunk:  99%|█████████▉| 1481/1490 [00:03<00:00, 461.54it/s, now=None]
100%|██████████| 7.75M/7.75M [2:06:26<00:00, 59.6MiB/s]

MoviePy - Done.
Moviepy - Writing video /content/self-introduction_final.mp4




t:  98%|█████████▊| 1066/1092 [00:04<00:00, 267.41it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/self-introduction.mp4, 691200 bytes wanted but 0 bytes read,at frame 1091/1092, at time 67.55/67.57 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+


100%|██████████| 7.75M/7.75M [2:06:30<00:00, 59.6MiB/s]

Moviepy - Done !
Moviepy - video ready /content/self-introduction_final.mp4
Final video saved to: /content/self-introduction_final.mp4
